In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost
from sklearn.cross_validation import train_test_split

C:\Users\satyam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_data=pd.read_csv("train_data_processed.csv")
test_data=pd.read_csv("test_data_processed.csv")

In [3]:
def build_features(data):
    data = data[data["Open"] != 0]
    data = data[data["Sales"] >0]
    features=['Store','CompetitionDistance','Promo','Promo2','SchoolHoliday','StoreType','Assortment','StateHoliday','DayOfWeek',
              'Month', 'Day', 'Year', 'WeekOfYear','CompetitionOpenSinceMonth','PromoOpen','IsPromoMonth','Customers']
    return data,features
train_data,features=build_features(train_data)

In [4]:
xg_model=xgboost.XGBRegressor()
xg_model.fit(train_data[features].values,train_data['Sales'].values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [5]:
predictions=xg_model.predict(test_data[features].values)

In [7]:
result1=test_data[['Id']]
result1["SalesX"]=predictions
result1.to_csv("Submission11.csv")

C:\Users\satyam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

In [10]:
params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.3,
          "max_depth": 10,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1,
          "seed": 1301
          }
num_boost_round = 300

In [32]:
X_train, X_valid = train_test_split(train_data, test_size=0.012, random_state=10)

In [33]:
y_train = np.log1p(X_train.Sales)
y_valid = np.log1p(X_valid.Sales)

AttributeError: 'DataFrame' object has no attribute 'Sales'

In [16]:
dtrain = xgboost.DMatrix(X_train[features], y_train)
dvalid = xgboost.DMatrix(X_valid[features], y_valid)

In [20]:
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgboost.train(params, dtrain, num_boost_round, evals=watchlist, \
  early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)

[0]	train-rmse:5.79341	eval-rmse:5.79751	train-rmspe:0.996844	eval-rmspe:0.996856
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 100 rounds.
[1]	train-rmse:4.05724	eval-rmse:4.06034	train-rmspe:0.981926	eval-rmspe:0.981972
[2]	train-rmse:2.84597	eval-rmse:2.84849	train-rmspe:0.939258	eval-rmspe:0.939355
[3]	train-rmse:1.99521	eval-rmse:1.99702	train-rmspe:0.859748	eval-rmspe:0.859918
[4]	train-rmse:1.39998	eval-rmse:1.40126	train-rmspe:0.748099	eval-rmspe:0.74831
[5]	train-rmse:0.983923	eval-rmse:0.984881	train-rmspe:0.620301	eval-rmspe:0.620516
[6]	train-rmse:0.696142	eval-rmse:0.697063	train-rmspe:0.494205	eval-rmspe:0.49451
[7]	train-rmse:0.496972	eval-rmse:0.497926	train-rmspe:0.383573	eval-rmspe:0.384009
[8]	train-rmse:0.361087	eval-rmse:0.362154	train-rmspe:0.294871	eval-rmspe:0.295496
[9]	train-rmse:0.263963	eval-rmse:0.264713	train-rmspe:0.225451	eval-rmspe:0.226041
[10]	train-rmse:0.203546	e

[96]	train-rmse:0.053878	eval-rmse:0.059384	train-rmspe:0.053584	eval-rmspe:0.059672
[97]	train-rmse:0.053749	eval-rmse:0.059344	train-rmspe:0.053454	eval-rmspe:0.059636
[98]	train-rmse:0.053635	eval-rmse:0.059277	train-rmspe:0.053343	eval-rmspe:0.059568
[99]	train-rmse:0.05354	eval-rmse:0.059217	train-rmspe:0.053245	eval-rmspe:0.059496
[100]	train-rmse:0.053351	eval-rmse:0.05906	train-rmspe:0.053059	eval-rmspe:0.05934
[101]	train-rmse:0.053234	eval-rmse:0.058984	train-rmspe:0.052944	eval-rmspe:0.059273
[102]	train-rmse:0.053169	eval-rmse:0.058947	train-rmspe:0.052878	eval-rmspe:0.059235
[103]	train-rmse:0.053065	eval-rmse:0.058871	train-rmspe:0.05278	eval-rmspe:0.059156
[104]	train-rmse:0.052907	eval-rmse:0.058779	train-rmspe:0.05262	eval-rmspe:0.05906
[105]	train-rmse:0.052776	eval-rmse:0.058733	train-rmspe:0.052487	eval-rmspe:0.059012
[106]	train-rmse:0.052718	eval-rmse:0.058707	train-rmspe:0.052431	eval-rmspe:0.058985
[107]	train-rmse:0.052592	eval-rmse:0.058639	train-rmspe:0.05230

[192]	train-rmse:0.046681	eval-rmse:0.055814	train-rmspe:0.046476	eval-rmspe:0.056044
[193]	train-rmse:0.04662	eval-rmse:0.055824	train-rmspe:0.046417	eval-rmspe:0.056055
[194]	train-rmse:0.046559	eval-rmse:0.055814	train-rmspe:0.046359	eval-rmspe:0.056044
[195]	train-rmse:0.046464	eval-rmse:0.055745	train-rmspe:0.046267	eval-rmspe:0.055976
[196]	train-rmse:0.046443	eval-rmse:0.05574	train-rmspe:0.046246	eval-rmspe:0.055971
[197]	train-rmse:0.046373	eval-rmse:0.055685	train-rmspe:0.04618	eval-rmspe:0.055919
[198]	train-rmse:0.046303	eval-rmse:0.055657	train-rmspe:0.046111	eval-rmspe:0.055889
[199]	train-rmse:0.046239	eval-rmse:0.055621	train-rmspe:0.04605	eval-rmspe:0.055848
[200]	train-rmse:0.046185	eval-rmse:0.055616	train-rmspe:0.046004	eval-rmspe:0.055843
[201]	train-rmse:0.046142	eval-rmse:0.0556	train-rmspe:0.045961	eval-rmspe:0.055824
[202]	train-rmse:0.046112	eval-rmse:0.055589	train-rmspe:0.045931	eval-rmspe:0.055811
[203]	train-rmse:0.046055	eval-rmse:0.055587	train-rmspe:0.0

[288]	train-rmse:0.042408	eval-rmse:0.054588	train-rmspe:0.04228	eval-rmspe:0.054738
[289]	train-rmse:0.042382	eval-rmse:0.054568	train-rmspe:0.042255	eval-rmspe:0.054719
[290]	train-rmse:0.042341	eval-rmse:0.054555	train-rmspe:0.042217	eval-rmspe:0.054705
[291]	train-rmse:0.042295	eval-rmse:0.054549	train-rmspe:0.042174	eval-rmspe:0.054697
[292]	train-rmse:0.042257	eval-rmse:0.054556	train-rmspe:0.042135	eval-rmspe:0.054705
[293]	train-rmse:0.042209	eval-rmse:0.05453	train-rmspe:0.042087	eval-rmspe:0.054679
[294]	train-rmse:0.042174	eval-rmse:0.054516	train-rmspe:0.042051	eval-rmspe:0.054665
[295]	train-rmse:0.042141	eval-rmse:0.054517	train-rmspe:0.042017	eval-rmspe:0.054666
[296]	train-rmse:0.042112	eval-rmse:0.054518	train-rmspe:0.041989	eval-rmspe:0.054668
[297]	train-rmse:0.042065	eval-rmse:0.054494	train-rmspe:0.041942	eval-rmspe:0.054643
[298]	train-rmse:0.041995	eval-rmse:0.054472	train-rmspe:0.041874	eval-rmspe:0.054616
[299]	train-rmse:0.04198	eval-rmse:0.054477	train-rmspe:

In [28]:
dtest = xgboost.DMatrix(test_data[features])
test_probs = gbm.predict(dtest)

In [30]:
result = pd.DataFrame({"Id": test_data["Id"], 'Sales': np.expm1(test_probs)})

In [31]:
result.to_csv("Submission12.csv")